In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 049694889102.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker images

In [86]:
!docker build -t 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo -f Dockerfile.train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 303B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.3s (10/10) FINISHED                               docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 303B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [ ]:
!docker images

In [87]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/cancer_data.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-11-04-16-04-49-684
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials fou

##### ESTIMATOR FIT STARTED
 Container lqauol4l3e-algo-1-6gg2y  Creating
 Container lqauol4l3e-algo-1-6gg2y  Created
Attaching to lqauol4l3e-algo-1-6gg2y
lqauol4l3e-algo-1-6gg2y  | 2024-11-04 16:04:51.009760: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
lqauol4l3e-algo-1-6gg2y  | 2024-11-04 16:04:51.013925: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
lqauol4l3e-algo-1-6gg2y  | 2024-11-04 16:04:51.026185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
lqauol4l3e-algo-1-6gg2y  | WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
lqauol4l3e-algo-1-6gg2y  | E0000 00:00:1730736291.046428       1 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register facto

INFO:sagemaker.local.image:===== Job Complete =====


lqauol4l3e-algo-1-6gg2y exited with code 0
Aborting on container exit...
 Container lqauol4l3e-algo-1-6gg2y  Stopping
 Container lqauol4l3e-algo-1-6gg2y  Stopped
##### ESTIMATOR FIT COMPLETED


In [80]:
!tar -xzf data/output/output.tar.gz

In [88]:
!tar -xzf ./data/output/model.tar.gz

In [89]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-training ######################
account: 049694889102 ######################
region: us-east-1 ######################
fullname: 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-pluto-2 ######################
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:049694889102:repository/tensorflow-training",
            "registryId": "049694889102",
            "repositoryName": "tensorflow-training",
            "repositoryUri": "049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training",
            "createdAt": 1730731234.555,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}


In [90]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  049694889102.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [91]:
!docker images

REPOSITORY                                                                  TAG          IMAGE ID       CREATED          SIZE
049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            mondo        8a3c62945c8a   6 minutes ago    8.59GB
<none>                                                                      <none>       4e4e3fe2a497   7 minutes ago    8.59GB
<none>                                                                      <none>       f9938f154dea   11 minutes ago   8.59GB
<none>                                                                      <none>       f21fc6c41401   13 minutes ago   8.59GB
<none>                                                                      <none>       6dc8cb2969fd   15 minutes ago   8.59GB
<none>                                                                      <none>       3a572a7d63b5   17 minutes ago   8.59GB
<none>                                                                      <none>       bf62765ccf46   25

In [ ]:
!docker tag ciao:mondo 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

In [ ]:
!docker images

In [92]:
!docker push 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

The push refers to repository [049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]

a51471d4: Preparing 
7650c473: Preparing 
7a6ebfb3: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
16f1a7a: Pushed   2.248GB/2.217GBPushing  385.4MB/1.042GBPushing  498.4MB/2.455GBPushing  1.503GB/2.455GBPushing    204MB/731.1MBPushing  1.727GB/2.217GBPushing  1.847GB/2.217GBPushing  1.918GB/2.217GBmondo: digest: sha256:632c993

In [94]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsrizzoli-flaviomanna/data/output/'
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsrizzoli-flaviomanna/data/input/cancer_data.csv')
print('##### ESTIMATOR FIT COMPLETED')

INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-11-04-16-16-03-117


##### ESTIMATOR FIT STARTED
2024-11-04 16:16:05 Starting - Starting the training job...
2024-11-04 16:16:29 Starting - Preparing the instances for training......
2024-11-04 16:17:32 Downloading - Downloading the training image...............
2024-11-04 16:19:59 Training - Training image download completed. Training in progress....2024-11-04 16:20:29.676618: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 16:20:29.680563: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-04 16:20:29.692757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730737229.712580      12 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already bee

In [ ]:
!pip3 install tensorflow pandas numpy matplotlib seaborn scikit-learn 

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler    
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
!pip3 freeze > requirements.txt